除了正则表达式之外，还有一些非常好用的信息筛选的工具，比如XPath表达式、Beautiful Soup等等。XPath表达式常用语法：

- / 逐层提取
- text() 提取标签下面的文本

- //标签名** 提取所有名为**的标签

- //标签名[@属性='属性值'] 提取属性为XX的标签

- @属性名 代表取某个属性值


In [20]:
import lxml

In [3]:
import urllib
from urllib import request

In [4]:
data_jd = request.urlopen("https://www.jd.com").read().decode("utf-8","ignore")
len(data_jd) # 判断有没有读取到数据

109390

In [7]:
import re
pat = "<title>(.*?)</title>" #正则表达式
re.compile(pat,re.S).findall(data_jd)

['京东(JD.COM)-正品低价、品质保障、配送及时、轻松购物！']

In [9]:
# 爬到内存中
data_csdn = request.urlopen("https://www.csdn.net").read().decode("utf-8","ignore")
re.compile(pat,re.S).findall(data_csdn)

['CSDN-专业IT技术社区']

In [12]:
# 直接将爬取的数据保存到硬盘中
url_jd = "https://www.jd.com"
save_path = "D:/Project/05 Python/jd.html"
request.urlretrieve(url_jd, filename = save_path)

('D:/Project/05 Python/jd.html', <http.client.HTTPMessage at 0x252b7a07048>)

In [13]:
#浏览器伪装
opener = request.build_opener() #创建opener对象
user_agent_key = "User-Agent"
user_agent_value = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
user_agent_info = (user_agent_key, user_agent_value)
opener.addheaders = [user_agent_info] #修改头信息
request.install_opener(opener) #将opener对象设置为全局有效
data = request.urlopen(url_jd).read().decode("utf-8","ignore")
len(data)

109679

In [18]:
#用户代理池
# 多个浏览器的标识构成用户代理池，在使用时随机的选择一个去访问，以防被反爬
import random
user_agent_pools = [
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
]

user_agent_key = "User-Agent"
user_agent_value = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
user_agent_info = (user_agent_key, user_agent_value)

url_jd = "https://www.jd.com"


def user_agent_func(pools, user_agent):
    opener = request.build_opener()
    pool_choice = random.choice(pools)
    user_agent = ("User-Agent",pool_choice)
    opener.addheaders = [user_agent]
    request.install_opener(opener)
    print("当前使用的用户代理为：",pool_choice)

# 每爬两次次换一次用户代理
for i in range(0,6):
    if (i%1 == 0):
        user_agent_func(user_agent_pools,user_agent_info)
    data_jd = urllib.request.urlopen(url_jd).read().decode("utf-8","ignore")
    print(len(data_jd))

当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7
109098
当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5
109098
当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14
109098
当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7
109098
当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5
109098
当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, l

In [19]:
# 分析流程：
## 1.取什么数据？
## 内容字段：<div class="main-text">
'''
<div class="content">
<span>


我姐跟我开视频聊天，抱怨她儿子：“你不知道，自从在家上网课，他每天太累了，累的我都心疼！”我大外甥马上过来跟我撒娇：“小姨，我妈都说我累，你是不是得奖励奖励我？”我姐瞬间把手机放下，我盯着天花板听着那边我姐揍娃的动静，一边揍还一边叨叨：“你可不就累坏了么，一整天不是找课本就是找本子，还间歇性找笔找眼镜，还想要奖励……”

</span>

</div>
'''
## 不同页码的网页地址及其规律：
# 2：https://www.qiushibaike.com/text/page/2/
## 构造网址

total_page = 1
for i in range(0,total_page):
    user_agent_func(user_agent_pools,user_agent_info)
    page_url = "https://www.qiushibaike.com/text/page/"+str(i+1)+"/"
    try:
        data = request.urlopen(page_url).read().decode("utf-8","ignore")
        pat = '<div class="content">.*?<span>(.*?)</span>.*?</div>'
        rst = re.compile(pat, re.S).findall(data)
        for j in range(0, len(rst)):
            print(rst[j])
            print("------")
    except Exception as error:
        pass
    

当前使用的用户代理为： Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9



老家院子有棵杨 桃 树，搬家后回老家少了，那年我爸带我们回去准备把树砍掉，怕时间久了落叶什么的没人打扫太脏。<br/>​我哭着抱住杨 桃 树，跟我爸说:杨桃树是我的小伙伴，从小它陪着我长大，它会开花结果给我杨 桃吃，还可以乘凉 荡 秋 千，我不想您砍掉它。<br/>我爸突然红了眼:这样说来，我也是它陪着长大的呢，但我想起小时候一闯祸你爷爷就把我绑在​这棵树上揍我，我一点都不心疼……


------



跟同事暧昧好久了，他虽然明显表示对我的好感，可迟迟不肯表白。<br/>今天我特意起了个大早做了份爱心早餐，想送给他吃顺便问问他的想法。哪成想到了公司却看到他跟别的女生打情骂俏，“腾”一下火就上来了，却又暗自窃喜没有表白。<br/>他看到我来了，过来跟我说话，眼尖的看到便当，问我:这该不会是给我吃的吧？<br/>我笑嘻嘻的回他:喂狗的！


------



几天没有回家，发现老爸憔悴了，郁郁寡欢的，我忙问怎么了，老爸叹气说，家里大家都嫌弃他，没人陪他看电视！我忙说我不嫌弃，我陪他看！<br/>晚上快十二点了，收拾下准备睡觉，老爸神秘兮兮地来叫我，说让我陪他看电视，我没多想，就去客厅，发现客厅里灯都关着，只有电视机开着，冒着诡异的光，音乐也特瘆得慌，我鸡皮疙瘩都起了，问老爸这是看啥！老爸诡异地笑：恐怖片！<br/>我听着那吓人的音乐，转头就要回房间，老爸一把拉住我，瞪着我说：白天谁说陪我看电视的？说话要算数！你敢跑我就打死你！


------



「转」<br/>悠悠白云悠悠过，涛涛江水涛涛流。<br/>英雄一去不复还，徒留遗恨业未就。<br/>化做龟蛇镇江汉，再做雷火驱鬼魔。<br/>唯愿中华万世安，清风陪我听凯歌。<br/>致敬，我的祖国！<br/>致敬，英雄的中国人民！


------



今天，不更糗事，不打游戏。沉痛悼念抗疫英烈、救火英雄和在疫情中牺牲的同胞。[比心][比心][比心]🙏🙏🙏


------



最合适的婚姻:一屋两人，三餐四季，狗在跑，我在闹，你在笑。夏天的傍晚一起散步